# LangGraph Sequential Tool Execution

Understanding how LangGraph executes dependent tools one after another.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand sequential execution** - When tasks have dependencies, the LLM executes tools one at a time, using results from earlier tools
2. **Recognize the sequential pattern** - Multiple separate AIMessages with tool_calls indicate sequential loops through the graph
3. **Verify data flow** - Examine how the LLM extracts values from tool results and uses them in subsequent tool calls

## 1. Environment Setup

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

## 2. Define Tools

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

## 3. Initialize LLM and Build Graph

In [ ]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """LLM node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

## 4. Query Analysis

Notice the dependency indicators: "**then**" and "**that amount**" signal the second task depends on the first.

In [ ]:
# Sequential execution test: dependent tasks
state = {
    "messages": [
        HumanMessage(content="Convert 1000 USD to EUR, then calculate the EMI for that amount in EUR at 7% for 48 months")
    ]
}

print("Query Analysis:")
print("=" * 80)
print(f"Query: {state['messages'][0].content}")
print("\nDependency indicators in query:")
print("  • 'then' - suggests order matters")
print("  • 'that amount' - refers to previous result")
print("  • 'in EUR' - must know converted amount first")
print("=" * 80)

## 5. Execute and Analyze

In [ ]:
# Execute the graph
result = app.invoke(state)

print("Execution Complete!")
print("=" * 80)
print(f"Total messages: {len(result['messages'])} (expected: 6 for sequential)")
print("=" * 80)

### Verify Sequential Tool Calls

Check for **multiple separate AIMessages** with tool_calls - this is the sequential pattern.

In [ ]:
# Verify sequential tool calls
tool_call_messages = [
    msg for msg in result['messages']
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls
]

print("SEQUENTIAL EXECUTION VERIFICATION")
print("=" * 80)
print(f"AIMessages with tool_calls: {len(tool_call_messages)}")

if len(tool_call_messages) > 1:
    print("\n🔄 CONFIRMED: Sequential execution detected!")
    print(f"   {len(tool_call_messages)} SEPARATE tool call requests")
else:
    print("\n⚠️  Single tool call message (might be parallel)")

print("\n" + "-" * 80)
print("Tool Call Sequence:")
print("-" * 80)
for i, msg in enumerate(tool_call_messages, 1):
    print(f"\n  Loop {i}: {msg.tool_calls[0]['name']}")
    print(f"    Args: {msg.tool_calls[0]['args']}")

### Complete Message Flow

Examine all messages to see the two loops through the graph.

In [ ]:
# Examine complete message flow
print("COMPLETE MESSAGE FLOW")
print("=" * 80)

loop_num = 0
for i, msg in enumerate(result["messages"], 1):
    print(f"\n{'─' * 80}")
    print(f"MESSAGE {i}: {type(msg).__name__}")
    print(f"{'─' * 80}")
    
    if isinstance(msg, HumanMessage):
        print(f"  👤 USER INPUT")
        print(f"  Content: {msg.content[:60]}..." if len(msg.content) > 60 else f"  Content: {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            loop_num += 1
            print(f"  🤖 AGENT: Loop {loop_num} - Calling tool")
            print(f"    Tool: {msg.tool_calls[0]['name']}")
            print(f"    Args: {msg.tool_calls[0]['args']}")
        else:
            print(f"  🤖 AGENT: Final synthesized response")
            preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
            print(f"  Content: {preview}")
            
    elif isinstance(msg, ToolMessage):
        print(f"  🔧 TOOL RESULT")
        print(f"  Tool Call ID: {msg.tool_call_id[:30]}...")
        lines = msg.content.split('\n')[:2]
        for line in lines:
            print(f"    {line}")

### Data Flow Verification

Verify the LLM correctly extracted the converted amount from the first tool and used it in the second tool call.

In [ ]:
# Verify data flow between tools
print("DATA FLOW VERIFICATION")
print("=" * 80)

# Get the currency conversion result
currency_result = result['messages'][2]  # First ToolMessage
print("STEP 1 - Currency Conversion Result:")
print(f"  {currency_result.content}")

# Get the EMI tool call
emi_call = result['messages'][3]  # Second AIMessage with tool_calls
print("\nSTEP 2 - EMI Calculator Called With:")
for key, value in emi_call.tool_calls[0]['args'].items():
    print(f"  {key}: {value}")

print("\n" + "-" * 80)
print("DEPENDENCY VERIFICATION:")
print("-" * 80)
extracted_principal = emi_call.tool_calls[0]['args'].get('principal')
extracted_currency = emi_call.tool_calls[0]['args'].get('currency')
print(f"  ✅ LLM extracted principal: {extracted_principal} {extracted_currency}")
print(f"  ✅ This matches the conversion result from Step 1!")

### Examine Individual Messages

In [ ]:
result['messages']

In [ ]:
dict(result['messages'][0])

In [ ]:
dict(result['messages'][1])

In [ ]:
dict(result['messages'][2])

In [ ]:
dict(result['messages'][3])

In [ ]:
dict(result['messages'][4])

In [ ]:
dict(result['messages'][5])

### Final Response

In [ ]:
# Show final response
print("\nFINAL RESPONSE:")
print("=" * 80)
print(result['messages'][-1].content)
print("=" * 80)

## 6. Streaming Execution

Observe the multiple loops in real-time through streaming.

In [ ]:
# Streaming view
state_stream = {
    "messages": [
        HumanMessage(content="Convert 1000 USD to EUR, then calculate the EMI for that amount in EUR at 7% for 48 months")
    ]
}

print("STREAMING EXECUTION")
print("=" * 80)
print("Watch the MULTIPLE loops in sequential execution...\n")

step_count = 0
loop_count = 0

for event in app.stream(state_stream):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: '{node_name}'")
        print("-" * 60)
        
        if "messages" in data:
            for msg in data["messages"]:
                if isinstance(msg, AIMessage):
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        loop_count += 1
                        print(f"  🔄 LOOP {loop_count}: Calling {msg.tool_calls[0]['name']}")
                        print(f"     Args: {msg.tool_calls[0]['args']}")
                    else:
                        print(f"  💬 Final response generated")
                        
                elif isinstance(msg, ToolMessage):
                    print(f"  ✅ Tool executed")
                    first_line = msg.content.split('\n')[0]
                    print(f"     Result: {first_line}")

print("\n" + "=" * 80)
print(f"Total steps: {step_count}")
print(f"Total loops (agent → tools): {loop_count}")
print("=" * 80)

## Conclusion

In this notebook, you learned:

✅ **Sequential execution** - When tasks have dependencies (e.g., "then", "that amount"), the LLM executes tools one at a time, using results from earlier tools to inform later ones

✅ **Sequential pattern** - Multiple separate AIMessages with tool_calls (e.g., AIMessage₁ → ToolMessage₁ → AIMessage₂ → ToolMessage₂) indicating 2+ loops through the graph, resulting in 6+ total messages

✅ **Data flow verification** - The LLM extracts values from ToolMessages (e.g., "920 EUR" from conversion) and uses them as parameters in subsequent tool calls (e.g., `principal=920, currency="EUR"` for EMI)

### Next Steps

Next, we'll explore **conversational context** where the agent maintains state across multiple user turns in a conversation.